In [27]:
import os

In [28]:
%pwd

'/Users/pavithrasenthilkumar/Documents/My_Codes/PROJECTS'

In [33]:
os.chdir("/Users/pavithrasenthilkumar/Documents/My_Codes/PROJECTS/Lung_Cancer_Classification_CT_Scan")

In [34]:
%pwd

'/Users/pavithrasenthilkumar/Documents/My_Codes/PROJECTS/Lung_Cancer_Classification_CT_Scan'

In [55]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float

In [56]:
from src.lungClassifier.constants import *
from src.lungClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [57]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    


    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE,
        )

        return training_config

In [58]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [ ]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path,
            compile=False
        )
        self.model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=self.config.params_learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

    def train_valid_generator(self):

        self.train_generator = tf.keras.utils.image_dataset_from_directory(
        directory=self.config.training_data,
        validation_split=0.2,
        subset="training",
        seed=42,
        image_size=self.config.params_image_size[:-1],
        batch_size=self.config.params_batch_size,
        label_mode="categorical"
    )

        self.valid_generator = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            validation_split=0.2,
            subset="validation",
            seed=42,
            image_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            label_mode="categorical"
        )

        # Normalize pixel values (rescale to 0-1)
        normalization_layer = tf.keras.layers.Rescaling(1./255)

        self.train_generator = self.train_generator.map(lambda x, y: (normalization_layer(x), y))
        self.valid_generator = self.valid_generator.map(lambda x, y: (normalization_layer(x), y))

        # Optional: speed up pipeline
        AUTOTUNE = tf.data.AUTOTUNE
        self.train_generator = self.train_generator.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
        self.valid_generator = self.valid_generator.cache().prefetch(buffer_size=AUTOTUNE)

    # @staticmethod
    # def save_model(path: Path, model: tf.keras.Model):
    #     model.save(path)
    
    def save_model(self, path: Path, model: tf.keras.Model):
        # Save full model as a single .h5 file
        # full_model_path = path.with_suffix(".h5")
        # model.save(full_model_path)

        model_json = model.to_json()
        with open("artifacts/training/model.json", "w") as json_file:
            json_file.write(model_json)

        # Save weights (MATCH the loading filename!)
        model.save_weights("artifacts/training/model.weights.h5")




    


    def train(self):
        # self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        # self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.steps_per_epoch = tf.data.experimental.cardinality(self.train_generator).numpy()
        self.validation_steps = tf.data.experimental.cardinality(self.valid_generator).numpy()      

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,  # example: Path("artifacts/training/model")
            model=self.model
        )

       

In [61]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-07-10 19:59:41,115: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-10 19:59:41,117: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-10 19:59:41,118: INFO: common: created directory at: artifacts]
[2025-07-10 19:59:41,119: INFO: common: created directory at: artifacts/training]
Found 302 files belonging to 2 classes.
Using 242 files for training.
Found 302 files belonging to 2 classes.
Using 60 files for validation.
16/16 ━━━━━━━━━━━━━━━━━━━━ 225s 14s/step - accuracy: 0.4542 - loss: 1.2667 - val_accuracy: 0.6500 - val_loss: 0.5680


ValueError: The filename must end in `.weights.h5`. Received: filepath=artifacts/training/model_weights.h5